# README.ipynb

Welcome to `labnotebook`. This is a docker-ized JupyterHub deployment that is setup to allow notebook kernels to run in Docker containers.

## Questions:

### How can I grant `admin` privliges to a different user?

Admin privlidges consist of administration on the Hub, and `sudo` root access on the Docker system. Once the user account has been created (they must successfully log in after being Authorized by the `admin` user) then the `admin` user can choose File > Hub Control Panel > Admin > Edit User and then check the `admin` option. This will give that user the ability to authorize or decline new users to use the Hub.

Separately, the `admin` user can grant sudo privliges to `anotheruser` in a terminal by running `sudo usermod -aG sudo anotheruser`. In order to get this to work `anotheruser` will need to restart their server from the Hub Control Panel.

### How can I generate a PDF of a notebook?

See [shared/samples/Exporting to PDF.ipynb](/hub/user-redirect/lab/tree/home/admin/shared/samples/Exporting%20to%20PDF.ipynb) for an example.

### Is there a way to user version control?

Yes, the excellent [jupyterlab-git](https://github.com/jupyterlab/jupyterlab-git) extension is installed. To connect to a remote repository (eg. GitHub), you'll need to provide SSH keys. These are automatically generated for users at `~/.ssh/id_ed25519.pub`.